In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


from skimage import io,transform
from PIL import Image 
import os
# from torch.utils.data import Dataset
from torchvision import transforms, utils,models
device = torch.device("cuda:0:" if torch.cuda.is_available() else "cpu")

In [2]:

resnet = models.resnet50(pretrained=True)

for param in resnet.parameters():
    param.requires_grad = False

n_inputs = resnet.fc.in_features

last_layer = nn.Linear(n_inputs, 128)
resnet.fc = last_layer
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:


class EmbeddingNet(nn.Module):
    def __init__(self,pretrained_net):
        super(EmbeddingNet, self).__init__()
        self.resnet = pretrained_net

    def forward(self, x):
        output = self.resnet(x)
        return output

    def get_embedding(self, x):
        return self.forward(x)



In [4]:
class TripletNet(nn.Module):
    '''
    input
    embedding net : ConvNet which takes torch.tensor input
     run parallel convnet for each batch
    '''
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2, x3):
        output1 = self.embedding_net(x1)
        output2 = self.embedding_net(x2)
        output3 = self.embedding_net(x3)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [5]:


embedding_net = EmbeddingNet(resnet)
model = TripletNet(embedding_net)



In [6]:
def load_weights(model=model):
    loaded_data = torch.load("./saved_models//saved_model_v3.pth",map_location={"cuda:0":"cpu"})
    model.load_state_dict(loaded_data)
    return model
embedding_net = load_weights(model)
embedding_net.eval()
print()

In [7]:
def transform_image(image):
    my_transforms = transforms.Compose([transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))])
    image_tensor = my_transforms(image).unsqueeze(0)
    return image_tensor

def prediction(image):
    tensor = transform_image(image=image)
    output = embedding_net.get_embedding(tensor).squeeze()
#     print(output.shape)
    return output

In [11]:
database_path = "./fashion-dataset/images/"

In [15]:
os.listdir(database_path).__len__()

44441

In [16]:
from tqdm import tqdm

In [23]:
def make_storage(database_path):
    storage = {}
#     for i in os.listdir(database_path):
#         path= database_path+"/"+i
#         pil = Image.open(path).convert("RGB")
    count =0
    for i,j,k in os.walk(database_path):
#         print(i,k)
        for inner in k:
            count+=1
            
            print(i+"/"+inner,count)
            path = i+"/"+inner
            pil = Image.open(path).convert("RGB")
            feature = prediction(pil).squeeze().cpu().detach().numpy()
            storage[path] = feature
    return storage
from numpy import dot
from numpy.linalg import norm
def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim    
    

In [ ]:
storage = make_storage(database_path)

./fashion-dataset/images//10000.jpg 1
./fashion-dataset/images//10001.jpg 2
./fashion-dataset/images//10002.jpg 3
./fashion-dataset/images//10003.jpg 4
./fashion-dataset/images//10004.jpg 5
./fashion-dataset/images//10005.jpg 6
./fashion-dataset/images//10006.jpg 7
./fashion-dataset/images//10007.jpg 8
./fashion-dataset/images//10008.jpg 9
./fashion-dataset/images//10009.jpg 10
./fashion-dataset/images//10010.jpg 11
./fashion-dataset/images//10011.jpg 12
./fashion-dataset/images//10012.jpg 13
./fashion-dataset/images//10013.jpg 14
./fashion-dataset/images//10014.jpg 15
./fashion-dataset/images//10015.jpg 16
./fashion-dataset/images//10016.jpg 17
./fashion-dataset/images//10017.jpg 18
./fashion-dataset/images//10018.jpg 19
./fashion-dataset/images//10020.jpg 20
./fashion-dataset/images//10021.jpg 21
./fashion-dataset/images//10022.jpg 22
./fashion-dataset/images//10023.jpg 23
./fashion-dataset/images//10024.jpg 24
./fashion-dataset/images//10025.jpg 25
./fashion-dataset/images//10026.jp

./fashion-dataset/images//10229.jpg 209
./fashion-dataset/images//10230.jpg 210
./fashion-dataset/images//10231.jpg 211
./fashion-dataset/images//10235.jpg 212
./fashion-dataset/images//10236.jpg 213
./fashion-dataset/images//10237.jpg 214
./fashion-dataset/images//10238.jpg 215
./fashion-dataset/images//10239.jpg 216
./fashion-dataset/images//10240.jpg 217
./fashion-dataset/images//10241.jpg 218
./fashion-dataset/images//10242.jpg 219
./fashion-dataset/images//10243.jpg 220
./fashion-dataset/images//10244.jpg 221
./fashion-dataset/images//10245.jpg 222
./fashion-dataset/images//10246.jpg 223
./fashion-dataset/images//10247.jpg 224
./fashion-dataset/images//10248.jpg 225
./fashion-dataset/images//10249.jpg 226
./fashion-dataset/images//10252.jpg 227
./fashion-dataset/images//10253.jpg 228
./fashion-dataset/images//10254.jpg 229
./fashion-dataset/images//10255.jpg 230
./fashion-dataset/images//10256.jpg 231
./fashion-dataset/images//10257.jpg 232
./fashion-dataset/images//10258.jpg 233


./fashion-dataset/images//10462.jpg 415
./fashion-dataset/images//10463.jpg 416
./fashion-dataset/images//10465.jpg 417
./fashion-dataset/images//10466.jpg 418
./fashion-dataset/images//10467.jpg 419
./fashion-dataset/images//10468.jpg 420
./fashion-dataset/images//10469.jpg 421
./fashion-dataset/images//10470.jpg 422
./fashion-dataset/images//10471.jpg 423
./fashion-dataset/images//10472.jpg 424
./fashion-dataset/images//10473.jpg 425
./fashion-dataset/images//10474.jpg 426
./fashion-dataset/images//10475.jpg 427
./fashion-dataset/images//10476.jpg 428
./fashion-dataset/images//10477.jpg 429
./fashion-dataset/images//10478.jpg 430
./fashion-dataset/images//10479.jpg 431
./fashion-dataset/images//10480.jpg 432
./fashion-dataset/images//10481.jpg 433
./fashion-dataset/images//10482.jpg 434
./fashion-dataset/images//10483.jpg 435
./fashion-dataset/images//10485.jpg 436
./fashion-dataset/images//10486.jpg 437
./fashion-dataset/images//10487.jpg 438
./fashion-dataset/images//10488.jpg 439


./fashion-dataset/images//10752.jpg 621
./fashion-dataset/images//10753.jpg 622
./fashion-dataset/images//10754.jpg 623
./fashion-dataset/images//10755.jpg 624
./fashion-dataset/images//10756.jpg 625
./fashion-dataset/images//10758.jpg 626
./fashion-dataset/images//10759.jpg 627
./fashion-dataset/images//10760.jpg 628
./fashion-dataset/images//10761.jpg 629
./fashion-dataset/images//10762.jpg 630
./fashion-dataset/images//10763.jpg 631
./fashion-dataset/images//10764.jpg 632
./fashion-dataset/images//10765.jpg 633
./fashion-dataset/images//10766.jpg 634
./fashion-dataset/images//10767.jpg 635
./fashion-dataset/images//10768.jpg 636
./fashion-dataset/images//10769.jpg 637
./fashion-dataset/images//10770.jpg 638
./fashion-dataset/images//10771.jpg 639
./fashion-dataset/images//10772.jpg 640
./fashion-dataset/images//10773.jpg 641
./fashion-dataset/images//10774.jpg 642
./fashion-dataset/images//10775.jpg 643
./fashion-dataset/images//10776.jpg 644
./fashion-dataset/images//10778.jpg 645


./fashion-dataset/images//11067.jpg 827
./fashion-dataset/images//11069.jpg 828
./fashion-dataset/images//11070.jpg 829
./fashion-dataset/images//11071.jpg 830
./fashion-dataset/images//11072.jpg 831
./fashion-dataset/images//11073.jpg 832
./fashion-dataset/images//11075.jpg 833
./fashion-dataset/images//11076.jpg 834
./fashion-dataset/images//11077.jpg 835
./fashion-dataset/images//11078.jpg 836
./fashion-dataset/images//11079.jpg 837
./fashion-dataset/images//11080.jpg 838
./fashion-dataset/images//11081.jpg 839
./fashion-dataset/images//11082.jpg 840
./fashion-dataset/images//11083.jpg 841
./fashion-dataset/images//11084.jpg 842
./fashion-dataset/images//11085.jpg 843
./fashion-dataset/images//11086.jpg 844
./fashion-dataset/images//11087.jpg 845
./fashion-dataset/images//11088.jpg 846
./fashion-dataset/images//11089.jpg 847
./fashion-dataset/images//11090.jpg 848
./fashion-dataset/images//11091.jpg 849
./fashion-dataset/images//11092.jpg 850
./fashion-dataset/images//11093.jpg 851


...

./fashion-dataset/images//33906.jpg 20080
./fashion-dataset/images//33907.jpg 20081
./fashion-dataset/images//33909.jpg 20082
./fashion-dataset/images//3391.jpg 20083
./fashion-dataset/images//33910.jpg 20084
./fashion-dataset/images//33911.jpg 20085
./fashion-dataset/images//33912.jpg 20086
./fashion-dataset/images//33913.jpg 20087
./fashion-dataset/images//33914.jpg 20088
./fashion-dataset/images//33915.jpg 20089
./fashion-dataset/images//33916.jpg 20090
./fashion-dataset/images//33917.jpg 20091
./fashion-dataset/images//33918.jpg 20092
./fashion-dataset/images//33919.jpg 20093
./fashion-dataset/images//33920.jpg 20094
./fashion-dataset/images//33921.jpg 20095
./fashion-dataset/images//33922.jpg 20096
./fashion-dataset/images//33923.jpg 20097
./fashion-dataset/images//33924.jpg 20098
./fashion-dataset/images//33925.jpg 20099
./fashion-dataset/images//33926.jpg 20100
./fashion-dataset/images//33928.jpg 20101
./fashion-dataset/images//33929.jpg 20102
./fashion-dataset/images//3393.jpg 

./fashion-dataset/images//34083.jpg 20276
./fashion-dataset/images//34084.jpg 20277
./fashion-dataset/images//34085.jpg 20278
./fashion-dataset/images//34087.jpg 20279
./fashion-dataset/images//34088.jpg 20280
./fashion-dataset/images//34089.jpg 20281
./fashion-dataset/images//3409.jpg 20282
./fashion-dataset/images//34090.jpg 20283
./fashion-dataset/images//34091.jpg 20284
./fashion-dataset/images//34092.jpg 20285
./fashion-dataset/images//34094.jpg 20286
./fashion-dataset/images//34095.jpg 20287
./fashion-dataset/images//34096.jpg 20288
./fashion-dataset/images//34098.jpg 20289
./fashion-dataset/images//34099.jpg 20290
./fashion-dataset/images//3410.jpg 20291
./fashion-dataset/images//34100.jpg 20292
./fashion-dataset/images//34101.jpg 20293
./fashion-dataset/images//34102.jpg 20294
./fashion-dataset/images//34103.jpg 20295
./fashion-dataset/images//34104.jpg 20296
./fashion-dataset/images//34105.jpg 20297
./fashion-dataset/images//34106.jpg 20298
./fashion-dataset/images//34107.jpg 

./fashion-dataset/images//34298.jpg 20472
./fashion-dataset/images//34299.jpg 20473
./fashion-dataset/images//34300.jpg 20474
./fashion-dataset/images//34301.jpg 20475
./fashion-dataset/images//34302.jpg 20476
./fashion-dataset/images//34303.jpg 20477
./fashion-dataset/images//34304.jpg 20478
./fashion-dataset/images//34307.jpg 20479
./fashion-dataset/images//34308.jpg 20480
./fashion-dataset/images//34309.jpg 20481
./fashion-dataset/images//3431.jpg 20482
./fashion-dataset/images//34310.jpg 20483
./fashion-dataset/images//34311.jpg 20484
./fashion-dataset/images//34313.jpg 20485
./fashion-dataset/images//34314.jpg 20486
./fashion-dataset/images//34315.jpg 20487
./fashion-dataset/images//34316.jpg 20488
./fashion-dataset/images//34317.jpg 20489
./fashion-dataset/images//34319.jpg 20490
./fashion-dataset/images//3432.jpg 20491
./fashion-dataset/images//34320.jpg 20492
./fashion-dataset/images//34321.jpg 20493
./fashion-dataset/images//34322.jpg 20494
./fashion-dataset/images//34323.jpg 

./fashion-dataset/images//34513.jpg 20668
./fashion-dataset/images//34514.jpg 20669
./fashion-dataset/images//34515.jpg 20670
./fashion-dataset/images//34516.jpg 20671
./fashion-dataset/images//34517.jpg 20672
./fashion-dataset/images//34518.jpg 20673
./fashion-dataset/images//34519.jpg 20674
./fashion-dataset/images//3452.jpg 20675
./fashion-dataset/images//34520.jpg 20676
./fashion-dataset/images//34521.jpg 20677
./fashion-dataset/images//34522.jpg 20678
./fashion-dataset/images//34523.jpg 20679
./fashion-dataset/images//34524.jpg 20680
./fashion-dataset/images//34525.jpg 20681
./fashion-dataset/images//34526.jpg 20682
./fashion-dataset/images//34527.jpg 20683
./fashion-dataset/images//34528.jpg 20684
./fashion-dataset/images//34529.jpg 20685
./fashion-dataset/images//34531.jpg 20686
./fashion-dataset/images//34532.jpg 20687
./fashion-dataset/images//34533.jpg 20688
./fashion-dataset/images//34534.jpg 20689
./fashion-dataset/images//34535.jpg 20690
./fashion-dataset/images//34536.jpg

./fashion-dataset/images//34774.jpg 20865
./fashion-dataset/images//34775.jpg 20866
./fashion-dataset/images//34776.jpg 20867
./fashion-dataset/images//34777.jpg 20868
./fashion-dataset/images//34778.jpg 20869
./fashion-dataset/images//34779.jpg 20870
./fashion-dataset/images//3478.jpg 20871
./fashion-dataset/images//34780.jpg 20872
./fashion-dataset/images//34781.jpg 20873
./fashion-dataset/images//34782.jpg 20874
./fashion-dataset/images//34785.jpg 20875
./fashion-dataset/images//34348.jpg 20876
./fashion-dataset/images//34369.jpg 20877
./fashion-dataset/images//34391.jpg 20878
./fashion-dataset/images//34411.jpg 20879
./fashion-dataset/images//34435.jpg 20880
./fashion-dataset/images//34455.jpg 20881
./fashion-dataset/images//34479.jpg 20882
./fashion-dataset/images//34511.jpg 20883
./fashion-dataset/images//3453.jpg 20884
./fashion-dataset/images//3455.jpg 20885
./fashion-dataset/images//34568.jpg 20886
./fashion-dataset/images//34587.jpg 20887
./fashion-dataset/images//34605.jpg 2

In [ ]:
import pickle    
with open('data.pickle', 'wb') as handle:
    pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
query_image = "./img/WOMEN/Blouses_Shirts/id_00003838/04_3_back.jpg"
pil_q = Image.open(query_image).convert("RGB")
query = prediction(pil_q).squeeze().cpu().detach().numpy()

In [ ]:
def make_search(storage,query):
    distance = {}
    for i,feature in storage.items():
        print(i)
        distance[i] = cosine_similarity(feature,query)
    return distance
        

In [ ]:
distance =make_search(storage,query)

In [ ]:
va =[]
for i,value in distance.items():
    if value>0.9:
        print(value,i)
        va.append({"image":Image.open(i).convert("RGB"),"score":value})


In [ ]:
import pickle
# with open('data.pickle', 'wb') as handle:
#     pickle.dump(storage, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)


In [ ]:
newlist=sorted(va, key = lambda k:k['score'], reverse=True)